# Signature extraction demonstration -- based on Groningen case

Description: Extract 10 SAR signautres including 'VV amplitude (linear), VH amplitude (linear), VV interferometric phase (radians), VV coherence, Intensity summation, Intensity difference (dual-pol difference), Intensity ratio (dual-pol power ratio), Cross-pol correlation coefficient, Cross-pol cross product,  and Entropy'. Four layers, 'Buildings, Railways, Water, and Roads', derived from TOP10NL dataset, are also included.

Authors: Anurag Kulshrestha, Xu Zhang, Ling Chang

Institution: University of Twente

Email: alignsar.project@gmail.com

1. Import required packages

In [ ]:
import os,sys
import matplotlib.pyplot as plt
import numpy as np
from scipy import signal
import numpy.ma as ma
import numpy.linalg as LA
import math

2. Define functions

In [ ]:
def get_p_value(eigen_full):
    '''

    :param eigen_full:
    :return:
    '''
    lamb1=eigen_full[...,1]
    lamb2=eigen_full[...,0]
    #lamb3=eigen_full[:,:,0]
    lamb_sum=np.sum(eigen_full, axis=3)# axis=3 for full stack computation; axis=2 for one by one computation
    p1=lamb1/lamb_sum
    p2=lamb2/lamb_sum
    #p3=lamb3/lamb_sum
    #return np.dstack((p1,p2,p3))
    return [p1,p2]

def entropy_cal(eigen_full):
    '''

    :param eigen_full:
    :return:
    '''
    p=get_p_value(eigen_full)
    p0_log=np.log(p[0])/math.log(2)
    #plt.imshow(p[1], cmap='gray')
    #plt.show()
    p1_log=np.log(p[1])/math.log(2)
    #p2_log=np.log(p[2])/math.log(3)
    ent=-1*(p[0]*p0_log+p[1]*p1_log)#+p[2]*p2_log)
    return ent

def plot_clipped(arr, per_min, per_max):
    '''

    :param arr:
    :param per_min:
    :param per_max:
    :return:
    '''
    return np.clip(arr, *np.percentile(arr, (per_min, per_max)))

def co_pol_diff(cov_arr):
    return cov_arr[...,0,0]-cov_arr[...,1,1]

def co_pol_correlation(cov_arr):
    '''

    :param cov_arr:
    :return:
    '''
    return cov_arr[...,0,1]/np.sqrt(cov_arr[...,0,0]*cov_arr[...,1,1])

def kernal(window_size):
    '''

    :param window_size:
    :return:
    '''
    k=np.ones(window_size*window_size).reshape(window_size, window_size)
    normalize_k=k/(window_size**2)
    return normalize_k

def test_convolve_3d(arr, AVG_WIN_SIZE, dtype=np.complex64):
    '''
    arr has a 3rd dim and conv is done per element in the 3rd dimension
    kernal should be 2 dimensional
    :param arr:
    :param AVG_WIN_SIZE:
    :param dtype:
    :return:
    '''

    kern = kernal(AVG_WIN_SIZE)
    shp = arr.shape
    ker_shp = kern.shape
    grad = np.empty((shp[0]-ker_shp[0]+1,\
        shp[1]-ker_shp[1]+1,\
            shp[-1]), dtype = dtype)
    for i in range(shp[-1]):
        grad[...,i] = signal.convolve2d(arr[...,i], kern, boundary='symm', mode='valid')
    return grad


def get_avg_cov_mat(vv_arr_stack, hh_arr_stack, AVG_WIN_SIZE = 5, PADDING=True):
    '''

    :param vv_arr_stack:
    :param hh_arr_stack:
    :param AVG_WIN_SIZE:
    :param PADDING:
    :return:
    '''

    shp_MN = vv_arr_stack.shape[:-1]
    
    epochs = vv_arr_stack.shape[-1]
    
    shp_MN = np.array(shp_MN) - AVG_WIN_SIZE +1
    cov_arr = np.dstack((\
        test_convolve_3d(np.absolute(hh_arr_stack)**2, AVG_WIN_SIZE),\
            test_convolve_3d(hh_arr_stack* np.conj(vv_arr_stack), AVG_WIN_SIZE),\
                test_convolve_3d(vv_arr_stack* np.conj(hh_arr_stack), AVG_WIN_SIZE),\
                    test_convolve_3d(np.absolute(vv_arr_stack)**2, AVG_WIN_SIZE)))\
                        .reshape(shp_MN[0], shp_MN[1],4,epochs).transpose(0,1,3,2)\
                            .reshape(shp_MN[0], shp_MN[1],epochs,2,2)
                        
    if (PADDING & (AVG_WIN_SIZE%2 !=0)):
        cov_arr = np.pad(cov_arr, pad_width = ((AVG_WIN_SIZE//2, AVG_WIN_SIZE//2), (AVG_WIN_SIZE//2, AVG_WIN_SIZE//2), \
            (0,0),(0,0),(0,0)))
    
    print('cov_arr.shape', cov_arr.shape)
    return cov_arr



3. Load prepared input data

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

dir = '/home/xu/MyProj/AlignSAR/Anurag_jupyter_notebook/Fea_Extract_Spk_Filt'

# import np arrays, which incldue resampled vv and vh slc data, and vv interferogram and coherence maps.
vv_cpx_stack = np.load(os.path.join(dir, 'data', 'Jupyter_input_vv_cpx_stack.npy'))
vh_cpx_stack = np.load(os.path.join(dir, 'data', 'Jupyter_input_vh_cpx_stack.npy'))
vv_coh_stack = np.load(os.path.join(dir, 'data', 'Jupyter_input_vv_coh_stack.npy'))
vv_ifg_stack = np.load(os.path.join(dir, 'data', 'Jupyter_input_vv_ifg_stack.npy'))

4. Calculate signatures

In [ ]:
# calculate the covariance matirx for intensity and pol information calcualtion
cov_arr = get_avg_cov_mat(vv_cpx_stack, vh_cpx_stack,AVG_WIN_SIZE = 3, PADDING=True)

# VV amplitude (linear)
vv_amp = np.abs(vv_cpx_stack)

# VH amplitude (linear)
vh_amp = np.abs(vh_cpx_stack)

# VV ifg (radians)
vv_ifg = vv_ifg_stack

# VV coh
vv_coh = vv_coh_stack

# Intensity summation |S_vv|^2 + |S_vh|^2
intensity_sum = np.square(vv_amp) + np.square(vh_amp)

# Intensity difference (cross-pol difference) |S_vv|^2 - |S_vh|^2
intensity_diff = np.square(vv_amp) - np.square(vh_amp)

# Intensity ratio (cross-pol power ratio) |S_vv|^2 / |S_vh|^2
intensity_ratio = np.square(vv_amp) / np.square(vh_amp)

# cross-pol correlation coefficient
cross_pol_corr_coeff = np.absolute(co_pol_correlation(cov_arr))

# cross-pol cross product
cross_pol_cross_product = np.absolute(co_pol_diff(cov_arr))

# entropy
entropy = entropy_cal(np.sort(np.absolute(LA.eigvals(cov_arr)), axis = 3))  # change axis = 3 in line 19 in 'get_p_value' function for whole stack

5. Visualization

In [ ]:
epochs=7
for i in range(epochs):
    
    dates=['20220109', '20220121', '20220202', '20220214', '20220226', '20220310', '20220322']
    print(dates[i])
    
    plt.figure(figsize=(30,60))
    plt.subplot(521)
    plt.imshow(plot_clipped(vv_amp[:,:,i],0.5,99.5), cmap='gray',aspect='auto')
    #plt.colorbar(label='linear',orientation='horizontal')
    plt.title('VV amplitude (linear)', fontsize=26)

    plt.subplot(522)
    plt.imshow(plot_clipped(vh_amp[:,:,i],0.5,99.5), cmap='gray',aspect='auto')
    #plt.colorbar(label='linear',orientation='horizontal')
    plt.title('VH amplitude (linear)', fontsize=26)

    plt.subplot(523)
    plt.imshow(vv_ifg[:,:,i], cmap='jet',aspect='auto')
    # plt.colorbar(label='linear',orientation='horizontal')
    plt.title('VV interferometric phase (radians)', fontsize=26)

    plt.subplot(524)
    plt.imshow(vv_coh[:,:,i], cmap='gray',aspect='auto')
    # plt.colorbar(label='linear',orientation='horizontal')
    plt.title('VV coherence', fontsize=26)

    plt.subplot(525)
    plt.imshow(plot_clipped(intensity_sum[:,:,i],1,99), cmap='gray',aspect='auto')
    # plt.colorbar(label='linear',orientation='horizontal')
    plt.title('Intensity summation', fontsize=26)

    plt.subplot(526)
    plt.imshow(plot_clipped(intensity_diff[:,:,i],1,99), cmap='gray',aspect='auto')
    # plt.colorbar(label='linear',orientation='horizontal')
    plt.title('Intensity difference (dual-pol difference)', fontsize=26)

    plt.subplot(527)
    plt.imshow(plot_clipped(intensity_ratio[:,:,i],1,99), cmap='gray',aspect='auto')
    # plt.colorbar(label='linear',orientation='horizontal')
    plt.title('Intensity ratio (dual-pol power ratio)', fontsize=26)

    plt.subplot(528)
    plt.imshow(cross_pol_corr_coeff[:,:,i], cmap='gray',aspect='auto')
    # plt.colorbar(label='linear',orientation='horizontal')
    plt.title('Cross-pol correlation coefficient', fontsize=26)

    plt.subplot(529)
    plt.imshow(plot_clipped(cross_pol_cross_product[:,:,i],1,99), cmap='gray',aspect='auto')
    # plt.colorbar(label='linear',orientation='horizontal')
    plt.title('Cross-pol cross product', fontsize=26)

    plt.subplot(5,2,10)
    plt.imshow(entropy[:,:,i], cmap='gray',aspect='auto')
    # plt.colorbar(label='linear',orientation='horizontal')
    plt.title('Entropy', fontsize=26)

    plt.subplots_adjust(wspace=0.2, hspace=0.2)
    plt.show()
